In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget
import pprint
import seaborn as sns 
sns.set()
#sns.reset_orig


## Part 1: Getting the Data
##




In [2]:
#######################
import myauz
from myauz.myalpha_funcs import (
    read_data,
    persist_data,
    update_csv,
    compose_portfolio,
    retrieveDF,
    string2date,
    retrievePF,
    initialize_df,
    time_sleep,
    create_path_list,
    refresh_db,
    get_daily_symbol,
)

#import importlib
#importlib.reload(myauz.myalpha_funcs)

from myauz.myalpha_libs_universal import StocksDb

import pprint


from datetime import date
import datetime
import pandas as pd



In [3]:
alphaDB = StocksDb()
alphaDB.check_path()
print("\n", alphaDB.api_key_alpha)


api_key_alpha = alphaDB.api_key_alpha
root_path = alphaDB.path
print('root_path: ',root_path)

read_from_alphavantage = False
rename_column = True


usecols = ["timestamp", "adjusted_close"]
startd = "2010-01-01"
endd = "2020-09-17"

print('usecols:', usecols)
print('startd',startd)
print('endd',endd)


Directory  ../data  already exists

 77K8XPSR3XWWZJ80
root_path:  ..
usecols: ['timestamp', 'adjusted_close']
startd 2010-01-01
endd 2020-09-17


In [4]:
symbol_list = [
   "MSFT",
   "GOOG",    
   "DHR",
   "EMR",
   "ATR",
   "HXGBY",
   "NVS",
   "NVZMY",    
   "CL",
   "MCD",    
   "HEINY",
   "LDSVF",  
]

path_list = create_path_list(symbol_list, root_path)
pprint.pprint(path_list)

bool_list_all = [
   True, #"MSFT", microsoft    
   True, #"GOOG", alphabet google      
#  True, #"AMZN", amazon
   True, #"DHR", danaher
   True, #"EMR", emerson    
   True, #"ATR", aptargroup
   True, #"HXGBY", hexagon
   True, #"NVS", novartis
   True, #"NVZMY", novozymes    
   True, #"CL", colgate palmolive
   True, #"MCD", mcdonalds    
   True, #"HEINY", heineken
   True, #"LDSVF", lindt & spruengli    
]

bool_list_high_tech = [
   True, #"MSFT", microsoft    
   True, #"GOOG", alphabet google      
#  True, #"AMZN", amazon
   False, #"DHR", danaher
   False, #"EMR", emerson    
   False, #"ATR", aptargroup
   False, #"HXGBY", hexagon
   False, #"NVS", novartis
   False, #"NVZMY", novozymes    
   False, #"CL", colgate palmolive
   False, #"MCD", mcdonalds    
   False, #"HEINY", heineken
   False, #"LDSVF", lindt & spruengli    
]

bool_list_industry = [
   False, #"MSFT", microsoft    
   False, #"GOOG", alphabet google      
#  False, #"AMZN", amazon
   True, #"DHR", danaher
   True, #"EMR", emerson    
   True, #"ATR", aptargroup
   True, #"HXGBY", hexagon
   True, #"NVS", novartis
   True, #"NVZMY", novozymes    
   False, #"CL", colgate palmolive
   False, #"MCD", mcdonalds    
   False, #"HEINY", heineken
   False, #"LDSVF", lindt & spruengli    
]

bool_list_consumer = [
   False, #"MSFT", microsoft    
   False, #"GOOG", alphabet google      
#  False, #"AMZN", amazon
   False, #"DHR", danaher
   False, #"EMR", emerson    
   False, #"ATR", aptargroup
   False, #"HXGBY", hexagon
   False, #"NVS", novartis
   False, #"NVZMY", novozymes    
   True, #"CL", colgate palmolive
   True, #"MCD", mcdonalds    
   True, #"HEINY", heineken
   True, #"LDSVF", lindt & spruengli    
]

colors = [
    'magenta',
    'lime',
#   'tab:orange',
    'blue',
    'black',
    'slategrey',
    'tab:brown',
    'tab:olive',
    'darkviolet',
    'red',
    'turquoise',
    'forestgreen',
    'gold',
]

color_dict = dict(zip(symbol_list, colors))
print('\n')
print(color_dict)


{'ATR': '../data/ATR/daily_ATR.csv',
 'CL': '../data/CL/daily_CL.csv',
 'DHR': '../data/DHR/daily_DHR.csv',
 'EMR': '../data/EMR/daily_EMR.csv',
 'GOOG': '../data/GOOG/daily_GOOG.csv',
 'HEINY': '../data/HEINY/daily_HEINY.csv',
 'HXGBY': '../data/HXGBY/daily_HXGBY.csv',
 'LDSVF': '../data/LDSVF/daily_LDSVF.csv',
 'MCD': '../data/MCD/daily_MCD.csv',
 'MSFT': '../data/MSFT/daily_MSFT.csv',
 'NVS': '../data/NVS/daily_NVS.csv',
 'NVZMY': '../data/NVZMY/daily_NVZMY.csv'}


{'MSFT': 'magenta', 'GOOG': 'lime', 'DHR': 'blue', 'EMR': 'black', 'ATR': 'slategrey', 'HXGBY': 'tab:brown', 'NVS': 'tab:olive', 'NVZMY': 'darkviolet', 'CL': 'red', 'MCD': 'turquoise', 'HEINY': 'forestgreen', 'LDSVF': 'gold'}


In [5]:
# filter columns(symbols) in dataframe  by bool_list(symbols = True - this is the _included list)
def pf_filtered(_pf, _startd, _included):
    if not (_startd):
        _startd = '2020-01-01'
    if _included is []:
        return _pf
    else: 
        return _pf.loc[startd:,_included]

from itertools import compress
def symbol_list_filtered(_symbol_list,_included):
    #filter the symbol_list
    #symbol_list_filtered = [i for indx,i in enumerate(symbol_list) if included[indx]]
    return list(compress(_symbol_list, _included))

def determine_earliest_notation(_pf):
    """
    input:
    ------
         portfolio - columns = symbol  - import also symbol_list
    
    output:
    -------
         dataframe with one row containing earliest notation in time of symbol in portfolio
    
    """
    _pf_tmp = _pf.copy()
    _pf_tmp.fillna(0, inplace=True)
    _symbol_list = list(_pf_tmp.columns)
    _df_aux_0 = _pf_tmp.iloc[0]
    
    for symbol in _symbol_list:
        _df_aux_0[symbol] = (_pf_tmp[_pf_tmp[symbol] > 0].iloc[0]).loc[symbol]  
    return _df_aux_0

In [6]:
print('root_path:',root_path)

root_path: ..


In [7]:
### create dataframe with portfolio adjusted close
usecols = ["timestamp", "adjusted_close"]
path_list = create_path_list(symbol_list, root_path)
pprint.pprint(path_list)
pf = retrievePF(symbol_list, path_list, startd, endd, usecols, rename_column)
print(pf.index.name)
pf.index.rename('DATE', inplace=True)
print(pf.index.name)
pf.fillna(method='ffill', inplace=True)
pf

{'ATR': '../data/ATR/daily_ATR.csv',
 'CL': '../data/CL/daily_CL.csv',
 'DHR': '../data/DHR/daily_DHR.csv',
 'EMR': '../data/EMR/daily_EMR.csv',
 'GOOG': '../data/GOOG/daily_GOOG.csv',
 'HEINY': '../data/HEINY/daily_HEINY.csv',
 'HXGBY': '../data/HXGBY/daily_HXGBY.csv',
 'LDSVF': '../data/LDSVF/daily_LDSVF.csv',
 'MCD': '../data/MCD/daily_MCD.csv',
 'MSFT': '../data/MSFT/daily_MSFT.csv',
 'NVS': '../data/NVS/daily_NVS.csv',
 'NVZMY': '../data/NVZMY/daily_NVZMY.csv'}
None
DATE


,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF
DATE,,,,,,,,,,,,
2010-01-04,24.1710,NaN,27.4647,31.2824,30.6109,NaN,35.5451,19.9437,32.1313,45.7426,NaN,2096.4120
2010-01-05,24.1788,NaN,27.4318,31.3329,30.5771,NaN,35.5113,20.0248,32.3912,45.3929,NaN,2078.6300
2010-01-06,24.0304,NaN,27.4830,31.6144,31.0417,NaN,35.3830,19.4935,32.3175,44.7736,NaN,2096.4120
2010-01-07,23.7820,NaN,27.7096,31.6144,31.0417,NaN,35.0722,19.7186,32.1895,45.1014,NaN,2096.4120
2010-01-08,23.9445,NaN,28.0969,31.7948,31.0670,NaN,35.3559,19.3675,31.6116,45.0577,NaN,2015.9248
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-10,205.3700,1532.02,203.0900,67.5100,117.1900,73.4300,88.5200,63.7300,76.4100,217.2400,45.445,8559.3500
2020-09-11,204.0300,1520.72,204.8700,67.9200,118.3600,74.6600,89.8800,65.5900,76.7500,218.0000,45.420,8559.3500
2020-09-14,205.4100,1519.28,207.9300,68.1900,119.0800,75.4694,90.0300,65.2750,77.1500,220.5600,45.990,8588.5000


In [8]:
df_returns = (pf/pf.shift(1))-1
#ret = pf.pct_change(1)
pprint.pprint(df_returns.head())

print('___________________')

df1_returns = pf_filtered(df_returns, startd, bool_list_high_tech)


print('\n high_tech:')
#df1_returns = (pf1/pf1.shift(1))-1
pprint.pprint(df1_returns.head(3))


print('\n industry:')
#df2_returns = (pf2/pf2.shift(1))-1
df2_returns = pf_filtered(df_returns, startd, bool_list_industry)
pprint.pprint(df2_returns.head(3))


print('\n consumer: ')
#df3_returns = (pf3/pf3.shift(1))-1
df3_returns = pf_filtered(df_returns, startd, bool_list_consumer)
pprint.pprint(df3_returns.head(3))

                MSFT  GOOG       DHR       EMR       ATR  HXGBY       NVS  \
DATE                                                                        
2010-01-04       NaN   NaN       NaN       NaN       NaN    NaN       NaN   
2010-01-05  0.000323   NaN -0.001198  0.001614 -0.001104    NaN -0.000951   
2010-01-06 -0.006138   NaN  0.001866  0.008984  0.015194    NaN -0.003613   
2010-01-07 -0.010337   NaN  0.008245  0.000000  0.000000    NaN -0.008784   
2010-01-08  0.006833   NaN  0.013977  0.005706  0.000815    NaN  0.008089   

               NVZMY        CL       MCD  HEINY     LDSVF  
DATE                                                       
2010-01-04       NaN       NaN       NaN    NaN       NaN  
2010-01-05  0.004066  0.008089 -0.007645    NaN -0.008482  
2010-01-06 -0.026532 -0.002275 -0.013643    NaN  0.008555  
2010-01-07  0.011547 -0.003961  0.007321    NaN  0.000000  
2010-01-08 -0.017806 -0.017953 -0.000969    NaN -0.038393  
___________________

 high_tech:
       

In [9]:
df_cum_returns = initialize_df(symbol_list, startd, endd)
#print(df_cum_returns.head())
#print(df_returns)

df_cum_returns = (1+df_returns).cumprod()
###############achtung dataframe wird auf naechtes datum ohne NaN gesetzt
#df_cum_returns.dropna(inplace=True)
#################
df_cum_returns.head()
#df_cum_returns.head()

,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF
DATE,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,1.000323,NaN,0.998802,1.001614,0.998896,NaN,0.999049,1.004066,1.008089,0.992355,NaN,0.991518
2010-01-06,0.994183,NaN,1.000666,1.010613,1.014073,NaN,0.995440,0.977426,1.005795,0.978816,NaN,1.000000
2010-01-07,0.983906,NaN,1.008917,1.010613,1.014073,NaN,0.986696,0.988713,1.001811,0.985982,NaN,1.000000
2010-01-08,0.990629,NaN,1.023019,1.016380,1.014900,NaN,0.994677,0.971109,0.983826,0.985027,NaN,0.961607


In [10]:
invested_dict = {
    "MSFT": float(1297.59),
    "GOOG": float(2111),
    "DHR": float(3168.48),
    "EMR": float(418.10),
    "ATR": float(289.49),
    "HXGBY": float(1060.39),
    "NVS": float(404.42),
    "NVZMY": float(1049.95),
    "CL": float(390.03),
    "MCD": float(872.35),  
    "HEINY": float(575.60),     
    "LDSVF": float(6942.25),   
}





def calc_weights(_invested_dict):
    
    _invested_list = list(_invested_dict.values())
    _invested_array =np.array(_invested_list)
    _total_investment=np.sum(_invested_array)
    #print("total investment: ")
    #pprint.pprint(_total_investment)
    _weights_dict = {}
    for symbol, investment in _invested_dict.items():
        _weights_dict[symbol] = investment / _total_investment
        
    return _total_investment, _weights_dict

total_investment, weights_dict = calc_weights(invested_dict)

print("total investment:",total_investment)

weight_sum = 0
for symbol, weight in weights_dict.items():
    pprint.pprint((symbol, weight))
    weight_sum += weight

print(weight_sum)

"""
weights = [
   .1,  #"MSFT",
   .1,  #"GOOG",    
   .1, #"DHR",
   .1,  #"EMR",
   .1,  #"ATR",
   .1,  #"HXGBY",
   .1,  #"NVS",
   .1,  #"NVZMY",    
   .1,  #"CL",
   .05,  #"MCD",    
   .03,  #"HEINY",
   .02,  #"LDSVF",  
]
"""



total investment: 18579.65
('MSFT', 0.0698393134423953)
('GOOG', 0.1136189325417863)
('DHR', 0.17053496702036905)
('EMR', 0.022503114967181836)
('ATR', 0.01558102547679854)
('HXGBY', 0.057072657450490186)
('NVS', 0.021766825532235537)
('NVZMY', 0.05651075235539959)
('CL', 0.020992322245036905)
('MCD', 0.04695190705960554)
('HEINY', 0.030980131487945144)
('LDSVF', 0.373648050420756)
0.9999999999999998


'\nweights = [\n   .1,  #"MSFT",\n   .1,  #"GOOG",    \n   .1, #"DHR",\n   .1,  #"EMR",\n   .1,  #"ATR",\n   .1,  #"HXGBY",\n   .1,  #"NVS",\n   .1,  #"NVZMY",    \n   .1,  #"CL",\n   .05,  #"MCD",    \n   .03,  #"HEINY",\n   .02,  #"LDSVF",  \n]\n'

In [11]:
#ensure that weights vector is in correct order as symbol_list
wlist = []
for symbol in symbol_list:
    print(symbol)
    wlist.append(weights_dict[symbol])

print('wlist',wlist)
#weights = np.array(list(weights_dict.values()))
weights=np.array(wlist)

print('\n',weights.sum())


pf_cum_ret = df_cum_returns * weights * total_investment
pf_cum_ret.fillna(method='ffill', inplace=True, axis=1)
pf_cum_ret['TOTAL_POS'] = pf_cum_ret.sum(axis=1)

print('\n _________________________________________________________________________________')

pf_cum_ret = pf_cum_ret[1:]

pf_cum_ret['TOTAL_POS']
pf_cum_ret.to_csv('../data/pf_cum_ret.csv')

MSFT
GOOG
DHR
EMR
ATR
HXGBY
NVS
NVZMY
CL
MCD
HEINY
LDSVF
wlist [0.0698393134423953, 0.1136189325417863, 0.17053496702036905, 0.022503114967181836, 0.01558102547679854, 0.057072657450490186, 0.021766825532235537, 0.05651075235539959, 0.020992322245036905, 0.04695190705960554, 0.030980131487945144, 0.373648050420756]

 0.9999999999999999

 _________________________________________________________________________________


In [12]:
plt.figure()
plt.title("total position of portfolio")
# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')





pf_cum_ret['TOTAL_POS'].plot(color=['royalblue'], figsize=(16,10), linewidth=1.25);


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
pf_cum_ret.head()
color_list = [color_dict.get(x, _) for x in pf_cum_ret.columns]

#pf_cum_ret.fillna(method='ffill', inplace=True)
pf_cum_ret[symbol_list].plot(color=color_list,figsize=(16,10), linewidth=1.25);


plt.title('all stocks in portfolio')
# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid( which='minor', linestyle=':', linewidth='0.5', color='gray')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Sharpe Ratio

In [14]:
pf_cum_ret['DAILY_RETURN']= pf_cum_ret['TOTAL_POS'].pct_change(1)

print('standard dev: ',pf_cum_ret['DAILY_RETURN'].std() )
print('mean: ', pf_cum_ret['DAILY_RETURN'].mean() )
      
cum_return = 100 * ( ( pf_cum_ret['TOTAL_POS'][-1] - pf_cum_ret['TOTAL_POS'][0] ) / pf_cum_ret['TOTAL_POS'][0] )
print('accumulated return: ',cum_return)


#calculate sharpe ratio assume risk-free rate to be zero
sharp_ratio = pf_cum_ret['DAILY_RETURN'].mean() / pf_cum_ret['DAILY_RETURN'].std()

# annualize Sharpe Ratio 
# annual sharpe ratio (asr) should be above 1 


print('\nannual sharp ration should be above 1')
sharp_ratio_annual = (252**0.5)*sharp_ratio

print('sharp_ratio: ', sharp_ratio)

print('annual_sharp_ratio: ',sharp_ratio_annual)

              



standard dev:  0.011708102183207445
mean:  0.0006636282742239644
accumulated return:  397.2765179206907

annual sharp ration should be above 1
sharp_ratio:  0.056681113970442204
annual_sharp_ratio:  0.8997847895993937


In [15]:
plt.figure()
plt.title("total position of portfolio")
# Turn on the minor TICKS, which are required for the minor GRID
plt.minorticks_on()
# Customize the major grid
plt.grid(which='major', linestyle='-', linewidth='0.5', color='dimgray')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')

pf_cum_ret['DAILY_RETURN'].plot(kind='kde', figsize=(10,8), xlim=[-.05,.05]);
pf_cum_ret


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF,TOTAL_POS,DAILY_RETURN
DATE,,,,,,,,,,,,,,
2010-01-05,1298.008733,1298.008733,3164.684474,418.774950,289.170350,289.170350,404.035435,1054.219566,393.184830,865.680926,865.680926,6883.365063,17223.984336,NaN
2010-01-06,1290.042064,1290.042064,3170.591189,422.537294,293.564114,293.564114,402.575682,1026.248907,392.290213,853.870352,853.870352,6942.250000,17231.446345,0.000433
2010-01-07,1276.707020,1276.707020,3196.733021,422.537294,293.564114,293.564114,399.039505,1038.099453,390.736468,860.121775,860.121775,6942.250000,17250.181559,0.001087
2010-01-08,1285.430630,1285.430630,3241.414096,424.948402,293.803378,293.803378,402.267347,1019.615549,383.721553,859.288379,859.288379,6675.717341,17024.729062,-0.013070
2010-01-11,1269.078559,1269.078559,3278.100338,431.410545,294.042643,294.042643,409.493298,1035.730397,382.027000,865.959360,865.959360,6675.717341,17070.640042,0.002697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-10,11025.032407,5791.093758,23429.587915,902.294293,1108.276238,4221.804965,1007.150308,3355.110311,927.512808,4142.950204,887.276113,28344.212654,85142.301973,-0.003824
2020-09-11,10953.096177,5748.379329,23634.938579,907.774084,1119.341032,4292.522930,1022.623923,3453.031308,931.639943,4157.444046,886.788009,28344.212654,85451.792013,0.003635
2020-09-14,11027.179757,5742.936074,23987.957138,911.382726,1126.150136,4339.058800,1024.330572,3436.447913,936.495395,4206.265407,897.916788,28440.742624,86076.863330,0.007315


# portfolio optimization
# - monte carlo
# - mathematical optimization by minimizing the negative sharpe ratio

In [16]:
pf


,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF
DATE,,,,,,,,,,,,
2010-01-04,24.1710,NaN,27.4647,31.2824,30.6109,NaN,35.5451,19.9437,32.1313,45.7426,NaN,2096.4120
2010-01-05,24.1788,NaN,27.4318,31.3329,30.5771,NaN,35.5113,20.0248,32.3912,45.3929,NaN,2078.6300
2010-01-06,24.0304,NaN,27.4830,31.6144,31.0417,NaN,35.3830,19.4935,32.3175,44.7736,NaN,2096.4120
2010-01-07,23.7820,NaN,27.7096,31.6144,31.0417,NaN,35.0722,19.7186,32.1895,45.1014,NaN,2096.4120
2010-01-08,23.9445,NaN,28.0969,31.7948,31.0670,NaN,35.3559,19.3675,31.6116,45.0577,NaN,2015.9248
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-10,205.3700,1532.02,203.0900,67.5100,117.1900,73.4300,88.5200,63.7300,76.4100,217.2400,45.445,8559.3500
2020-09-11,204.0300,1520.72,204.8700,67.9200,118.3600,74.6600,89.8800,65.5900,76.7500,218.0000,45.420,8559.3500
2020-09-14,205.4100,1519.28,207.9300,68.1900,119.0800,75.4694,90.0300,65.2750,77.1500,220.5600,45.990,8588.5000


In [17]:
# annual mean of returns as %
type(pf.pct_change(1).mean()*252)
(pf.pct_change(1).mean()*252).sort_values(ascending=False)


NVZMY    0.458078
MSFT     0.232228
HXGBY    0.217288
DHR      0.212016
GOOG     0.189193
MCD      0.167438
LDSVF    0.162788
ATR      0.146566
EMR      0.110479
NVS      0.104746
CL       0.098420
HEINY    0.078290
dtype: float64

In [18]:
# we use log returns for algorithms especially for normalizations
log_ret= np.log(pf/pf.shift(1))

log_ret.head()

,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF
DATE,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.000323,NaN,-0.001199,0.001613,-0.001105,NaN,-0.000951,0.004058,0.008056,-0.007674,NaN,-0.008518
2010-01-06,-0.006157,NaN,0.001865,0.008944,0.015080,NaN,-0.003619,-0.026890,-0.002278,-0.013737,NaN,0.008518
2010-01-07,-0.010391,NaN,0.008211,0.000000,0.000000,NaN,-0.008823,0.011481,-0.003969,0.007295,NaN,0.000000
2010-01-08,0.006810,NaN,0.013880,0.005690,0.000815,NaN,0.008056,-0.017966,-0.018116,-0.000969,NaN,-0.039149


In [19]:
(log_ret.mean()).sort_values(ascending=False)

MSFT     0.000794
DHR      0.000750
HXGBY    0.000660
GOOG     0.000615
MCD      0.000591
LDSVF    0.000517
ATR      0.000500
NVZMY    0.000444
NVS      0.000344
CL       0.000324
EMR      0.000292
HEINY    0.000224
dtype: float64

In [20]:
# annual variance
print((log_ret.var()*252).sort_values(ascending=False))

# annual standard deviation
print((log_ret.std()*252**0.5).sort_values(ascending=False))

NVZMY    0.582472
HXGBY    0.100507
EMR      0.074158
GOOG     0.068409
LDSVF    0.064470
MSFT     0.064336
DHR      0.045972
HEINY    0.043868
ATR      0.041000
MCD      0.036908
NVS      0.036088
CL       0.033524
dtype: float64
NVZMY    0.763199
HXGBY    0.317028
EMR      0.272319
GOOG     0.261551
LDSVF    0.253909
MSFT     0.253645
DHR      0.214411
HEINY    0.209446
ATR      0.202483
MCD      0.192115
NVS      0.189968
CL       0.183096
dtype: float64


In [21]:
log_ret.corr() 

,MSFT,GOOG,DHR,EMR,ATR,HXGBY,NVS,NVZMY,CL,MCD,HEINY,LDSVF
MSFT,1.000000,0.718073,0.560425,0.511176,0.482217,0.332624,0.461503,0.118294,0.430892,0.477656,0.355582,0.062328
GOOG,0.718073,1.000000,0.530903,0.463757,0.408554,0.358501,0.415018,0.236504,0.380295,0.446178,0.357936,0.062572
DHR,0.560425,0.530903,1.000000,0.593106,0.563707,0.282792,0.521688,0.161447,0.464379,0.483161,0.362590,0.080965
EMR,0.511176,0.463757,0.593106,1.000000,0.552261,0.368113,0.444582,0.145131,0.386550,0.489971,0.390651,0.068303
ATR,0.482217,0.408554,0.563707,0.552261,1.000000,0.273552,0.457889,0.140820,0.466164,0.449906,0.362175,0.058146
HXGBY,0.332624,0.358501,0.282792,0.368113,0.273552,1.000000,0.269167,0.198519,0.252642,0.287451,0.242151,0.125329
NVS,0.461503,0.415018,0.521688,0.444582,0.457889,0.269167,1.000000,0.157864,0.426866,0.398765,0.477068,0.113302
NVZMY,0.118294,0.236504,0.161447,0.145131,0.140820,0.198519,0.157864,1.000000,0.090308,0.084872,0.302331,0.018378
CL,0.430892,0.380295,0.464379,0.386550,0.466164,0.252642,0.426866,0.090308,1.000000,0.453284,0.383276,0.069918
MCD,0.477656,0.446178,0.483161,0.489971,0.449906,0.287451,0.398765,0.084872,0.453284,1.000000,0.324821,0.033148


In [27]:
plt.ioff(); fig = plt.figure(); plt.ioff()
log_ret.hist(bins=100, figsize=(12,12));
plt.figure()

<Figure size 432x288 with 0 Axes>